# Création d'un dataset d'évaluation pour la detection de marque, modèle et calibre.
Le dataset sera constitué de texte brut issu de photos d'armes avec comme label la marque/modèle calibres associés ainsi que la référence RGA

## Données sources
Nous nous baserons sur un extract de la base de données naturabuy car celle-ci contient des photos des armes ainsi que la référence RGA associé ce qui permettra de remonter au calibre et modèle

## Création du dataset
Pour chaque image du dataset naturabuy, nous allons lancer un modèle d'OCR (Paddle) afin de recupérer le texte brut
Ensuite il sera réalisé un matching entre le RGA et les données naturabuy pour recuperer marque modelèe et calibre

## nettoyage du dataset
Un nettoyage sera necessaire pour vérifier qu'il n'y a pas de pollution des données. exemples:
- filigrane (naturabuy)
- autre texte que l'arme (feuilles papier)
- photos sans armes
- armes sans marquages

Cette tâche de nettoyage sera assez chronophage car réalisée à la main
Le nettoyage pourrait être réalisé en amont pour economiser du temps de calcul et conserver uniquement les images contenant des armes avec marquages

Piste -> création d'un script d'affichage des images puis suppression de l'image si pas de texte de façon manuelle

In [3]:
from paddleocr import PaddleOCR,draw_ocr
import os
from PIL import Image
import pandas as pd
import numpy as np
#import pyiqa

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `french`, `german`, `korean`, `japan`
# to switch the language model in order.
OCR=PaddleOCR(det_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/detection', rec_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/recognition', cls_model_dir='C:/Users/aurelien.martinez/Documents/Basegun/3.OCR/PaddleModels/classification', use_angle_cls=True,show_log = False) # need to run only once to download and load model into memory

# inference on images

In [2]:
def get_text(results):
    """extracts raw text from PaddleOCR output
    Args:
        results: raw result from PaddleOCR

    Returns:
        text: A string with the text extracted from the image
    """
    text=" "
    for result in results:
        text=text+result[1][0]+" "
    return text.lower()
    
def imageprocessing(imglink):
    results = OCR.ocr(imglink, cls=True)
    if results!=[None]:#The results with recongition and detection confidence below 0.5 are filtered by paddle, the thresholds values can be changed
        res=get_text(results[0])
        return res.replace('naturabuy','')
    else:
        return ''

In [4]:
#Initialize dataframe
data = pd.read_csv('NBdata.csv',on_bad_lines='skip')
data=data.drop(["auction_id","cat_name","title","label"],axis=1)
data.replace(np.nan, '', inplace=True)
data=data.set_index('rga_ref')
for key in data:
    data[key+'text'] = ''

In [4]:
def processRow(data,i):
    for key in data:
        if data[key][i] != '' and not('text' in key):
            res=imageprocessing(data[key][i])
            data[key+'text'][i] = res
            os.remove("tmp.jpg")

In [9]:
from tqdm import tqdm
for i in tqdm(range(500)):
    try:
        processRow(data,i+2000)
    except:
        print('error')
        os.remove("tmp.jpg")

  0%|          | 0/500 [00:00<?, ?it/s]
61.7kiB [00:00, 1.40MiB/s]                 

62.1kiB [00:00, 1.83MiB/s]                 

71.3kiB [00:00, 1.82MiB/s]                 

58.1kiB [00:00, 2.21MiB/s]                 

39.2kiB [00:00, 1.94MiB/s]                 

38.4kiB [00:00, 4.68MiB/s]                 

40.6kiB [00:00, 13.6MiB/s]                 

40.2kiB [00:00, 3.73MiB/s]                 

43.7kiB [00:00, 1.99MiB/s]                 

40.1kiB [00:00, 1.90MiB/s]                 

45.3kiB [00:00, 2.03MiB/s]                 
  0%|          | 1/500 [00:09<1:23:05,  9.99s/it]
  0%|          | 0.00/1.00 [00:00<?, ?iB/s]
243kiB [00:00, 1.80MiB/s]                  

  0%|          | 0.00/1.00 [00:00<?, ?iB/s]
284kiB [00:00, 1.69MiB/s]                  

  0%|          | 0.00/1.00 [00:00<?, ?iB/s]
317kiB [00:00, 1.76MiB/s]                  

  0%|          | 0.00/1.00 [00:00<?, ?iB/s]
274kiB [00:00, 2.20MiB/s]                  

191kiB [00:00, 2.14MiB/s]                  

  0%|          

error



37.3kiB [00:00, 1.85MiB/s]                 
 14%|█▍        | 70/500 [12:38<38:27,  5.37s/it]

error



52.1kiB [00:00, 1.28MiB/s]                 

65.3kiB [00:00, 1.19MiB/s]                 

65.8kiB [00:00, 1.48MiB/s]                 

  0%|          | 0.00/1.00 [00:00<?, ?iB/s]
86.3kiB [00:00, 466kiB/s]                  

104kiB [00:00, 1.24MiB/s]                  

62.3kiB [00:00, 2.88MiB/s]                 

58.2kiB [00:00, 2.74MiB/s]                 

58.2kiB [00:00, 2.57MiB/s]                 

81.4kiB [00:00, 1.28MiB/s]                 
 14%|█▍        | 71/500 [12:47<46:14,  6.47s/it]
81.6kiB [00:00, 1.29MiB/s]                 

31.9kiB [00:00, 2.05MiB/s]                 

61.6kiB [00:00, 1.51MiB/s]                 
 14%|█▍        | 72/500 [12:52<41:45,  5.86s/it]

...

error



100kiB [00:00, 1.42MiB/s]                  
100%|█████████▉| 499/500 [16:52<00:00,  2.04it/s]

error



105kiB [00:00, 1.73MiB/s]                  
100%|██████████| 500/500 [16:52<00:00,  2.03s/it]

error


In [8]:
data.to_excel('outReadNB3.xlsx')

# Script tri

In [6]:
    from IPython.display import Image
    from IPython.core.display import HTML 

In [18]:
a=''
for i in range(10):
    for key in data:
        img=data[key][i]
        if img !='':
            display(Image(url= img))
            a=input()
            if a=='y':
                pass
            elif a=='stop':
                break
            else:
                data[key][i]=''
        else:
            pass
    if a=='stop':
        data.to_excel('datalabelized.xlsx')
        break
    else:
        pass

 y


 y


 stop


In [14]:
data

,file1,file2,file3,file4,file5,file6,file7,file8,file9,file10,...,file3text,file4text,file5text,file6text,file7text,file8text,file9text,file10text,file11text,file12text
rga_ref,,,,,,,,,,,,,,,,,,,,,
AN067,,,,,,,,,,,...,,,,,,,,,,
AG230,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,https://one.nbstatic.fr/uploaded/20080710/2971...,...,,,,,,,,,,
BS024,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,https://one.nbstatic.fr/uploaded/20081120/5564...,...,,,,,,,,,,
AU494,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,https://one.nbstatic.fr/uploaded/20090104/6800...,,,,...,,,,,,,,,,
BS505,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,https://one.nbstatic.fr/uploaded/20090709/1424...,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AN939,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,https://one.nbstatic.fr/uploaded/20231225/1130...,...,,,,,,,,,,
AZ087,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,https://one.nbstatic.fr/uploaded/20240119/1139...,...,,,,,,,,,,
AT500,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,https://one.nbstatic.fr/uploaded/20240123/1140...,,,...,,,,,,,,,,
